In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
energy_file = "Resources/energy.csv"
energy_df = pd.read_csv(energy_file)
energy_df.head()

,state,TotalC2010,TotalC2011,TotalC2012,TotalC2013,TotalC2014,TotalP2010,TotalP2011,TotalP2012,TotalP2013,...,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014
0,Alabama,1931522,1905207,1879716,1919365,1958221,1419613,1400108,1433370,1463647,...,1.165832,1.157861,-0.020443,-0.168414,0.396416,0.420102,1.011941,1.001333,1.562247,1.577963
1,Alaska,653221,653637,649341,621107,603119,1738207,1641980,1563102,1513859,...,3.203618,2.869760,-1.175137,-1.949571,-3.789313,-13.754494,0.948185,1.835376,-0.585695,-10.884734
2,Arizona,1383531,1424944,1395839,1414383,1422590,580948,617956,598039,594994,...,1.090035,1.091283,1.341472,-0.420875,-0.580562,-1.313050,2.317801,0.621971,0.509473,-0.221767
3,Arkansas,1120632,1122544,1067642,1096438,1114409,1247709,1391190,1472778,1432074,...,2.141877,2.129805,1.369514,5.131282,3.910476,6.280636,3.336628,7.155212,6.052353,8.410441
4,California,7760629,7777115,7564063,7665241,7620082,2532205,2634789,2334863,2390424,...,4.207353,4.177389,-1.162079,-1.173951,-1.341226,-0.830982,2.761377,2.772770,2.866127,3.346406


In [3]:
census_file = "Resources/state_census.csv"
census_df = pd.read_csv(census_file)
census_df.head()

,state,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,"4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185"
1,Alaska,"713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545"
2,Arizona,"6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717"
3,Arkansas,"2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804"
4,California,"37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223"


In [5]:
# Create a filtered dataframe from specific columns
energy_cols = ["state", "CoalC2014", "CoalPrice2014", "ElecC2014", "ElecPrice2014" ]
energy_transformed= energy_df[energy_cols].copy()

# Rename the column headers
energy_transformed = energy_transformed.rename(columns={"state": "us_states",
                                                        "CoalC2014": "coal_usage", 
                                                        "CoalPrice2014": "coal_price", 
                                                        "ElecC2014": "electric_usage", 
                                                        "ElecPrice2014": "electric_price"
                                                        })


energy_transformed.set_index("us_states", inplace=True)


energy_transformed.head()

,coal_usage,coal_price,electric_usage,electric_price
us_states,,,,
Alabama,575912,2.91,308765,27.20
Alaska,18225,4.87,21034,51.27
Arizona,447849,2.11,260328,29.83
Arkansas,339214,2.42,160638,23.19
California,39486,3.43,895939,44.49


### Transform census DataFrame

In [6]:
census_cols = ["state", "2014"]
census_transformed = census_df[census_cols].copy()

# Rename the column headers
census_transformed = census_transformed.rename(columns={"state": "us_states",
                                                        "2014": "population_2014"
                                                       })

# Set index
census_transformed.set_index("us_states", inplace=True)

census_transformed.head()

,population_2014
us_states,
Alabama,"4,841,799"
Alaska,"736,283"
Arizona,"6,730,413"
Arkansas,"2,967,392"
California,"38,596,972"


### Create database connection

In [7]:
connection_string = "postgres:Ovnmt8412$@localhost:5432/State_Energy"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# Confirm tables
engine.table_names()

['energy', 'census']

### Load DataFrames into database

In [9]:
energy_transformed.to_sql(name='energy', con=engine, if_exists='append', index=True)

In [10]:
census_transformed.to_sql(name='census', con=engine, if_exists='append', index=True)